In [ ]:
basePath='D:\\PM\\IV2A\\'
dbType='IV2A_Imagery'
random_seed = 32654

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## setup

In [2]:
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False


import tensorflow as tf
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)


import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


In [ ]:



#!pip uninstall tensorflow==2.2.0 --yes
!pip uninstall keras --yes

      #!pip install argcomplete
!pip install keras==2.2.4
#!pip install tensorflow==1.10



!pip install ipython-autotime
!pip install plot_keras_history

print('~'*50)


%tensorflow_version 1.10
import tensorflow
print(tensorflow.__version__)
print('~'*50)

print('ok')
 

In [ ]:
# %tensorflow_version 1.10
import tensorflow
print(tensorflow.__version__)

In [ ]:
#%load_ext autotime
#%unload_ext autotime


## Prepare Data 25920

In [ ]:


import scipy.io
import pandas as pd
import numpy as np
import pickle as pkl
import gc



def prepare_data():
  #~~~~~~~~~~~~~~~~~~~~~~~~~~~     prepare data   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  

  with open('D:\PM\MMCNN_2A_data_raw.pkl','rb') as f: #read
    _Data = pkl.load(f)
  
  print(_Data.shape)
  print('-------------------------------------------')


  with open('D:\PM\MMCNN_2A_label_raw.pkl','rb') as f: #read
    _Target = pkl.load(f)
  
  print(_Target.shape)
  print('-------------------------------------------')


  _Data = _Data.reshape( _Data.shape[0] , 1000 * 3 )

  print(' _Data shape is : ' , _Data.shape)
  print(' _Target shape is : ' , _Target.shape)
  

  return _Data, _Target



_Data, _Target= prepare_data()
print(' Load data ==> OK  ')

In [ ]:
from sklearn.model_selection import train_test_split

def split_and_prepare_Data(_Data, _Target):
  X_train, X_test, y_train, y_test = train_test_split( _Data, _Target , test_size=0.3 , random_state=random_seed  , shuffle=True)  #276159



  print("\x1b[31m  ....................................  \x1b[0m")
  print('X_train: ', X_train.shape)
  print('X_test: ', X_test.shape)
  print('y_train: ', y_train.shape)
  print('y_test: ', y_test.shape)
  print("\x1b[31m  ................. Total ...................  \x1b[0m")



  print()
  print("\x1b[31m  .................. Summary ..................  \x1b[0m")
  print('X_train: ', X_train.shape)
  print('X_test: ', X_test.shape)
  print('y_train: ', y_train.shape)
  print('y_test: ', y_test.shape)
 

  print("\x1b[31m  ....................................  \x1b[0m")



  import pickle as pkl
  with open(basePath+'Imagery_temp_X_test_IV2A.pkl','wb') as f: #write
    pkl.dump( X_test , f)

  with open(basePath +'Imagery_temp_y_test_IV2A.pkl','wb') as f: #write
    pkl.dump( y_test , f)

  del  X_test, y_test


  return X_train,y_train


X_train, y_train= split_and_prepare_Data(_Data, _Target)  



In [ ]:
del _Data, _Target

In [ ]:
print( X_train.shape)
print(y_train.shape)

In [ ]:
y_train2=[np.where(r==1)[0][0] for r in np.array( y_train ) ]

## Reshaper

In [3]:

from sklearn.base import BaseEstimator , TransformerMixin

class CustomLSTMReshaper(BaseEstimator , TransformerMixin):
    
     def fit(self,X,y=None):
        return self
    
    
     def transform(self, X, y=None):     
        return X.reshape(( len(X), 1000,3  ))



## Train Metrics

In [2]:
from keras import backend as K


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


# https://medium.com/@mostafa.m.ayoub/customize-your-keras-metrics-44ac2e2980bd
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred) #precision_m
    recall = recall_m(y_true, y_pred) #recall_m
    return 2*( (precision*recall)/(precision+recall+K.epsilon() )   )


def specificity_m(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())


# implement at: https://www.sabinasz.net/unbalanced-classes-machine-learning/
def sensitivity_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())    





## Test Metric

In [3]:

## Metric library
from imblearn.metrics import sensitivity_score ,specificity_score #geometric_mean_score

from sklearn.metrics import(
                            accuracy_score,
                            precision_score, 
                            recall_score,
                            f1_score,                                                      
                            make_scorer,                            
                            cohen_kappa_score
                            )


def make_experiments_for_tuner():
       
    accuracy = make_scorer(accuracy_score)
    precision = make_scorer(precision_score , average = 'macro' )
    recall = make_scorer(recall_score , average = 'macro')
    f1 = make_scorer(f1_score  , average = 'macro')
    sensitivity = make_scorer( sensitivity_score  , average = 'macro')     
    specificity = make_scorer( specificity_score , average = 'macro')
    kappa= make_scorer(cohen_kappa_score   )

    scores_for_grid = {
    "accuracy":accuracy,
    "precision":precision, 
    "recall":recall,
    "f1":f1,
    "sensitivity":sensitivity,
    "specificity":specificity,
    "kappa":kappa
    }
    
    return scores_for_grid


def get_experiments(true_label,pred_label):
    
     return  {
    "test_accuracy":accuracy_score(true_label,pred_label ),
    "test_precision":precision_score(true_label,pred_label , average='macro' ), 
    "test_recall":recall_score(true_label,pred_label , average='macro' ),
    "test_f1":f1_score(true_label,pred_label , average='macro' ),
    "test_specificity":specificity_score(true_label,pred_label , average='macro'),    
    "test_sensitivity":sensitivity_score(true_label,pred_label , average='macro'),
    "test_kappa":cohen_kappa_score(true_label,pred_label   )
     }



In [ ]:

## Metric library
from sklearn.metrics import(
                            accuracy_score,
                            precision_score, 
                            recall_score,
                            f1_score,                                                      
                            make_scorer,
                            cohen_kappa_score
                            )


def make_experiments_for_tuner_micro():
       
    accuracy = make_scorer(accuracy_score)
    precision = make_scorer(precision_score , average = 'micro' )
    recall = make_scorer(recall_score , average = 'micro')
    f1 = make_scorer(f1_score  , average = 'micro')
    sensitivity = make_scorer( sensitivity_score  , average = 'micro') 
    specificity = make_scorer( specificity_score , average = 'micro')
    kappa= make_scorer(cohen_kappa_score  )

    scores_for_grid = {
    "accuracy":accuracy,
    "precision":precision, 
    "recall":recall,
    "f1":f1,
    "sensitivity":sensitivity,
    "specificity":specificity,
    "kappa":kappa
    }
    
    return scores_for_grid


def get_experiments_micro(true_label,pred_label):
    
     return  {
    "test_accuracy":accuracy_score(true_label,pred_label ),
    "test_precision":precision_score(true_label,pred_label , average='micro' ), 
    "test_recall":recall_score(true_label,pred_label , average='micro' ),
    "test_f1":f1_score(true_label,pred_label , average='micro' ),
    "test_sensitivity":sensitivity_score(true_label,pred_label , average='micro'),
    "test_specificity":specificity_score(true_label,pred_label , average='micro'),
    "test_kappa":cohen_kappa_score(true_label,pred_label  )
     }

## model





### Time Distributed after LSTM



In [4]:
from keras.layers.core import Dense
from keras.layers.recurrent import LSTM 
from keras.layers import Flatten, Dropout,RepeatVector,Activation, Dense , CuDNNLSTM 
from keras.models import Sequential
from keras.optimizers import  Adamax
from keras.layers.wrappers import TimeDistributed
from keras.regularizers import l2,l1_l2


# https://medium.com/smileinnovation/how-to-work-with-time-distributed-data-in-a-neural-network-b8b39aa4ce00

def make_model_LSTM_distribute_dense( ):
  print('make_model_LSTM_distribute_dense in selected....')
  

  model = Sequential()




  from keras.layers import LeakyReLU
  #from multiplicative_lstm import MultiplicativeLSTM   # recurrent_dropout=0.2,
  model.add(  CuDNNLSTM(64,    kernel_regularizer= l2(0.001), return_sequences=True, input_shape=(1000, 3 ) )) #CuDNNLSTM
 
  model.add(Dropout(0.37))


  model.add(TimeDistributed( Dense( 64 )  )) 
 


  model.add(Flatten())
  model.add(Dense(128 ))
  
  model.add(Dropout(.3))

  # use N outputs
  model.add(Dense( 2, activation='sigmoid'))

  opt=Adamax(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-5) #*******************************

  model.compile(loss='binary_crossentropy', optimizer=opt ,  metrics= ['binary_accuracy'  , precision_m, recall_m, f1_m, sensitivity_m, specificity_m  ])
    

  return model  


In [ ]:
ss=make_model_LSTM_distribute_dense( )
ss.summary()

### Model_Maker

In [7]:
def Model_Maker( ):  
  

  return make_model_LSTM_distribute_dense() 

## Pipeline


In [ ]:

 
from imblearn.pipeline import Pipeline, make_pipeline
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
 
 

   
def make_pipeline():    

    main_pipeline = Pipeline(steps =[
        ('scaler',StandardScaler() ),           
        ('Reshaping' , CustomLSTMReshaper()),         #                                                              , callbacks=callbacks_list    shuffle=True,            , callbacks=[rlrp ]
        ('classifier', KerasClassifier(  build_fn=Model_Maker  , batch_size=512, verbose=1, epochs=1800    ))  # validation_split=0.3 
        #,('freeUpMemory', freeUp_Memory() )     , callbacks=[rlrp, lrm]             , use_multiprocessing=True , validation_data=( X_val, y_val)
    ],  verbose=True )
    
    return main_pipeline

## Visualization

In [11]:
from plot_keras_history import plot_history
import matplotlib.pyplot as plt

def Visualize_Result(mp , k, path ,  dbType): 

  #pip install plot_keras_history
  #https://pypi.org/project/plot-keras-history/

  estimator= mp['classifier'] 
  history=estimator.model.history.history


  # plot_history(history)
  # plt.show()

  plot_history(history, path= path + dbType +"_"+ "_fold("+ str(k) +").png" )
  plt.close()
  return 


# post_Visualization

In [12]:

from matplotlib import pyplot as pl
import numpy as np



def dict_mean(dict_list):
  mean_dict = {}
  std_dict = {}
  

  for key in dict_list[0].keys():
      mean_dict[key] = np.mean([d[key] for d in dict_list], axis=0)
      std_dict[key] = np.std([d[key] for d in dict_list], axis=0)
      
  return mean_dict, std_dict


j=0 

def Auto_All_History_plot(all_history, savepath , dbType):

  result=dict_mean(all_history )

  for val_metric_name in result[0]:
    
    if str( val_metric_name[0:4]) !='val_': # This is a condition for not calculating the results for both train and validation categories
        break                               # Calculations are performed for train category and evaluation is also considered for validation

    metric_name=str( val_metric_name[4:])


    mean=result[0][ metric_name ] # mean
    std=result[1][ metric_name ] #std


    val_mean=result[0][ val_metric_name ] # mean
    val_std=result[1][ val_metric_name ] #std


    pl.plot(mean ,color='blue' ) #color='red'
    pl.plot(val_mean ,color='red' )

    pl.fill_between(  range(len(mean)) , mean-std, mean + std , alpha=0.3, label='Train' ,color='blue' )
    pl.fill_between(  range(len(val_mean)) , val_mean-val_std, val_mean + val_std , alpha=0.3 , label='Validation' ,color='red' )

    if str( metric_name[-2:] )=='_m':
        metric_name=metric_name[:-2]
    

    plt.title( 'The Model '+metric_name+' Plot')
    plt.xlabel('Epoch')
    plt.ylabel( metric_name )

    plt.legend(loc='best')
    #pl.show()
    plt.savefig( savepath+dbType+ '_'+ metric_name+'.png')
    plt.close()

  return


# Test Model

In [13]:
# test Unseen data
import pickle as pkl

def LoadUnseenData():
  
  with open(basePath+'Imagery_temp_X_test_IV2A.pkl','rb') as f: #read
    X_test = pkl.load(f)
    
  #print(X_test.shape)
  

  with open(basePath+'Imagery_temp_y_test_IV2A.pkl','rb') as f: #read
    Y_test = pkl.load(f)
    
    
    # karimian -- 1400 01 27
    Y_test=[np.where(r==1)[0][0] for r in Y_test]
    
  #print(Y_test.shape)
  print('-------------------------------------------')

  return X_test, Y_test

# over_all_metrics

In [14]:
def over_all_metrics_stat(skf_metrics, savePath):
  # Take mean and std over all folds 
  average_metrics_all_folds = dict(pd.DataFrame(skf_metrics).mean())
  std_metrics_all_folds = dict(pd.DataFrame(skf_metrics).std())

  

  average_metrics_all_folds = {str( k[5:])+"_avg_folds":v for k,v in average_metrics_all_folds.items()}
  std_metrics_all_folds = {str( k[5:])+"_std_folds":v for k,v in std_metrics_all_folds.items()}

  print('_________ Training metric for all k-fold _________')
  print('average_metrics_all_folds')
  print(average_metrics_all_folds)
  print()
  print('std_metrics_all_folds')    
  print(std_metrics_all_folds)
  print()


  df_average_metrics_all_folds = pd.DataFrame([average_metrics_all_folds])
  df_std_metrics_all_folds = pd.DataFrame([std_metrics_all_folds])

  pd.concat([df_average_metrics_all_folds, df_std_metrics_all_folds ],axis=1 ).to_csv( savePath + dbType + '_UnSeen_data_metric_avg_std.csv')


  return     

    

# k-fold Writer

In [ ]:
from sklearn.model_selection import StratifiedKFold

n_splits=5
skf = StratifiedKFold(n_splits=n_splits, shuffle=False ) #,random_state=random_seed


    


folds = {}
count = 1
for train_index, val_index in skf.split(X_train, y_train2 ):
    folds['fold_{}'.format(count)] = {}
    folds['fold_{}'.format(count)]['train_index'] = train_index.tolist()
    folds['fold_{}'.format(count)]['val_index'] = val_index.tolist()
    count += 1
print(len(folds) == n_splits)#assert we have the same number of splits


#dump folds to json
# import json
# with open( basePath+'folds.json', 'w') as fp:
#     json.dump(folds, fp)    



In [ ]:
import pandas as pd
import gc
from keras.utils.np_utils import to_categorical 
import matplotlib.pyplot as plt
import pickle




i=1

for key, val in folds.items():


  print(key)
  print(i)

  train_index = val['train_index']
  val_index = val['val_index']


  
  skf_X_train, skf_X_val = (pd.DataFrame( X_train )).iloc[train_index], (pd.DataFrame( X_train)).iloc[val_index]
  skf_y_train, skf_y_val = (pd.DataFrame( y_train2 )).iloc[train_index], (pd.DataFrame( y_train2 )).iloc[val_index]



 
  with open( basePath + 'kfold_data_variable ('+ str(i) +').pkl', 'wb') as p: 
      pickle.dump([ skf_X_train, skf_X_val, skf_y_train, skf_y_val  ], p)


  print('_step_',i)
  i+=1
  

# k-fold Reader

In [15]:
# load previous config
import pickle as pkl
import pandas as pd


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
try:
  with open(basePath + dbType +'_Kfold_Imagery_all_history.pkl','rb') as f: #read
    all_history = pkl.load(f)

  with open(basePath + dbType+ '_UnSeen_data_metric_avg_std.pkl','rb') as f: #read
    skf_metrics = pkl.load(f)


except:
  print('var empty mod')
  all_history= []
  skf_metrics= []
 
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

var empty mod


# Main Excutation

In [23]:
print(len(all_history))

4


In [ ]:

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ RUN ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import pandas as pd
import gc
from keras.utils.np_utils import to_categorical 
import matplotlib.pyplot as plt
import pickle 


gc.collect()


i=1 # part number in k- fold [1,2,3,4,5]


with open( basePath + 'kfold_data_variable ('+ str( i) +').pkl' ,'rb') as p:   
    skf_X_train, skf_X_val, skf_y_train, skf_y_val  = pickle.load(p)

gc.collect()


print("============ Training on fold ({}) ==============".format(i))

#skf_y_val2=[np.where(r==1)[0][0] for r in np.array(skf_y_val) ]


skf_y_train2= to_categorical( skf_y_train , num_classes=2)
skf_y_val2= to_categorical( skf_y_val , num_classes=2)




pipeline = make_pipeline( ) 

# ------------------------
pipeline.named_steps['scaler'].fit(skf_X_train)
skf_X_val22=pipeline.named_steps['scaler'].transform(skf_X_val)
skf_X_val2=pipeline.named_steps['Reshaping'].transform(skf_X_val22)
# ------------------------





pipeline.fit(skf_X_train, skf_y_train2 , classifier__validation_data=( skf_X_val2, skf_y_val2)  )

gc.collect()
del skf_X_train, skf_y_train, skf_y_val, skf_y_train2, skf_y_val2, skf_X_val2, skf_X_val22, skf_X_val







X_test, Y_test =LoadUnseenData()

y_pred_val = pipeline.predict( X_test  )
del X_test
gc.collect()

print("_________________________________________________")
print()





print('___step___',i)
print()
print('Model Evaluation is in process...')
test_results = get_experiments( Y_test , y_pred_val)
print(test_results)
del Y_test , y_pred_val

gc.collect()

skf_metrics.append(test_results)



# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
all_history.append( pipeline['classifier'].model.history.history )
print('Model Training history has been store')
print()
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


Visualize_Result(pipeline , i, basePath ,   dbType )
print('Evaluation Resault has been Visualized')
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


# # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
del pipeline
gc.collect()

# # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


with open(basePath + dbType +'_Kfold_Imagery_all_history.pkl','wb') as f: #write
  pkl.dump( all_history , f)
print('Model Training history Resault has been wrote on file')


with open(basePath + dbType +'_UnSeen_data_metric_avg_std.pkl','wb') as f: #write
  pkl.dump( skf_metrics , f)
print('Model Evaliuation Resault has been wrote on file')  


print()



In [ ]:


over_all_metrics_stat(skf_metrics, basePath)


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
pd.DataFrame([all_history]).to_csv( basePath+ dbType+'_All_history.csv')

Auto_All_History_plot( all_history , basePath ,dbType)

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


df = pd.DataFrame( skf_metrics, columns=['test_accuracy', 'test_precision', 'test_recall' ,'test_f1' , 'test_specificity', 'test_sensitivity', 'test_kappa' ])
pd.DataFrame( df ).to_csv( basePath + dbType + '_All_UnseenData_Results_df.csv' , index=False )

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


print(' .: Model Training has been finished successfully :. ')



# save model 

In [ ]:


from keras.models import load_model
from sklearn.externals import joblib

...

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('estimator', KerasRegressor(build_model))
])

pipeline.fit(X_train, y_train)

# Save the Keras model first:
pipeline.named_steps['estimator'].model.save('keras_model.h5')

# This hack allows us to save the sklearn pipeline:
pipeline.named_steps['estimator'].model = None

# Finally, save the pipeline:
joblib.dump(pipeline, 'sklearn_pipeline.pkl')

del pipeline









# Load the pipeline first:
pipeline = joblib.load('sklearn_pipeline.pkl')

# Then, load the Keras model:
pipeline.named_steps['estimator'].model = load_model('keras_model.h5')

y_pred = pipeline.predict(X_test)


In [ ]:

# from sklearn.externals import joblib


# Save the Keras model first:
# mp.named_steps['classifier'].model.save('/content/drive/My Drive/SA_NTM_990922/EEG_Model_Classification_model.h5')

# This hack allows us to save the sklearn pipeline:
#pipeline.named_steps['estimator'].model = None

# Finally, save the pipeline:
#joblib.dump( mp , '/content/drive/My Drive/SA_NTM_990922/EEG_Model_Classification_pipeline.pkl')




In [ ]:
# import pickle as pkl
# with open('/content/drive/My Drive/SA_NTM_990922/history.pkl','wb') as f: #write
#     pkl.dump( d , f)


In [ ]:
  # import pickle as pkl
  # with open('/content/drive/My Drive/SA_NTM_990922/history.pkl','rb') as f: #read
  #   history = pkl.load(f)

In [ ]:
# load model & pipeline
# from keras.models import load_model
# from sklearn.externals import joblib

# from multiplicative_lstm import MultiplicativeLSTM


# # Load the pipeline first:
# pipeline = joblib.load('./EEG_Model_Classification_pipeline.pkl')

# # Then, load the Keras model:
# pipeline.named_steps['estimator'].model = load_model('./EEG_Model_Classification_model.h5')

# y_pred = pipeline.predict(~temp_X_test)


In [ ]:
#################################################################

# print('start data section') 
# gc.collect()                                                  
# X_train,X_test, Y_train, Y_test= Load_data()
# gc.collect()
# print('finish data section')




#y_train3 = tf.keras.utils.to_categorical(y_train2, 4)

# model=make_model(batch_size)
# #y_train3
# history=model.fit(X_train7, y_train, validation_split=0.28125, epochs=25, batch_size=batch_size, verbose=1 ) #, callbacks=[tensorboard_callback]




In [ ]:
print(history)


In [ ]:
del X_train7, y_train3

In [ ]:
with open('~temp_X_test_02.pkl','rb') as f: #read
  _X_test = pkl.load(f)
  
print(_X_test.shape)
print('-------------------------------------------')

In [ ]:
############################################################################

# Y_pred = mp.predict(X_test)
# test_results = get_experiments(Y_test,Y_pred)

estimator= mp['classifier'] 
d=estimator.model #.history.history

#estimator.model.history.history
#print(a.model.history.history)

import tensorflow as tf
Y_test2 = tf.keras.utils.to_categorical(Y_test, 2)

# حتمن باید استاندارد اسکالر هم اعمال شود
X_test3=X_test.reshape(( len(X_test) ,80,64 ))


print("Evaluation....")
score= d.evaluate(X_test3, Y_test2, batch_size=1024, verbose=1)                                           


print('Training results')
print('-------------------------------------------')
print('Training loss:', score[0])
print('Training accuracy:', score[1])
print('-------------------------------------------')    
print('*********** finish *************')



In [ ]:
print(Y_test.shape)